In [1]:
import torch 
import random 
import math
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import math 

In [2]:
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad = True 
x2 = torch.Tensor([0.0]).double() ; x2.requires_grad = True 
w1 = torch.Tensor([-3.0]).double() ; w1.requires_grad = True 
w2 = torch.Tensor([1.0]).double() ; w2.requires_grad = True 

b = torch.Tensor([6.88]).double() ; b.requires_grad = True 

n = x1*w1 + x2*w2 + b 
o = torch.tanh(n) 

print(o.data.item()) #returns item stripping tensor 
o.backward() 

print('---') 
print('x2', x2.grad.item()) 
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())


0.7064193777288968
---
x2 0.5009716627691181
w2 0.0
x1 -1.5029149883073543
w1 1.0019433255382362


In [3]:
# neural nets are basically a specific class of mathematical experesison 

In [4]:
torch.Tensor([[1,2,3], [4,5,6]])

# scalars in a compat representation 2 by 3 arrays 
# double to have a float64 instead of the instant 32 
# explicitly say the values needs gradient usually set to false 

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [5]:
# copying 

class Value: 

    def __init__(self, data, _children=(), _op='', label = ''): # self is a requirement for an inwards pass 
        self.data = data 
        self.grad = 0.0 
        self._backward = lambda: None 
        self._prev = set(_children) # set and convenience
        self._op = _op  
        self.label = label

    def __repr__(self): # official string 
        return f"Value(data = {self.data})" # repr function to return a string 
    def __add__(self, other): 
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other), '+')
        def _backward():
            self.grad += 1.0*out.grad 
            other.grad += 1.0*out.grad 
        out._backward = _backward 

        return out 
    def __neg__(self): 
        return self * -1 
    def __sub__(self, other): 
        return self + (-other)    
      
    def __mul__(self, other): 
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other), '*')
        def _backward():
            self.grad = other.data*out.grad 
            other.grad = self.data*out.grad 
        out._backward = _backward 
        return out 
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        def _backward():
            self.grad += out.grad*out.grad  
        out._backward = _backward 
        return out 
        
    
    def __rmul__(self, other): 
        return self * other 

    def __truediv__(self, other): 
        return self * other**-1    


    def __pow__(self, other): 
        assert isinstance (other, (int, float))
        out = Value (self.data**other, (self,), f'**(other)')   
        def backward(): 
            self.grad += other * self.data ** (other-1) * out.grad 
        out._backward = backward() 

        return out 


    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad = (1 - t**2 ) * out.grad 
        out._backward = _backward 

        return out
    
    def backward(self):
        topo = [] 
        visited = set() 
        def build_topo(v): 
            if v not in visited: 
                visited.add(v)
                for child in v._prev: 
                    build_topo(child)
            topo.append(v)
        build_topo(self)

        self.grad = 1.0 
        for node in reversed(topo): 
            node._backward() 

from graphviz import Digraph 

def trace(root): 
# builds a set of all nodges and edges in a graph
    nodes, edges = set(), set() 
    def build(v): 
        if v not in nodes: 
            nodes.add(v)
            for child in v._prev: 
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges 

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir':'LR'}) # left to right 

    nodes, edges = trace(root) 
    for n in nodes: 
        uld = str(id(n))

        dot.node(name = uld, label = "{ %s | data %.4f | grad %.4f}" % (n.label, n.data,n.grad), shape = 'record')
        if n._op:
            dot.node(name = uld + n._op, label = n._op)

            dot.edge(uld+n._op, uld)

    for n1, n2 in edges: 
        dot.edge(str(id(n1)), str(id(n2)) +n2._op)
    return dot

In [27]:


class Neuron: 
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1)) 
    
    def __call__(self, x): 
        
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b) 
        out = act.tanh() 
        return out  
# nin is number of inputs, nouts is number of neurons in a single layer 
    def parameters(self): 
        return self.w + [self.b]
class Layer: 
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x): 
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    def parameters(self): 
        return [p for neuron in self.neurons for p in neuron.parameters()]
class MLP: 
    def __init__(self, nin, nouts):
        sz = [nin] + nouts 
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    
    def __call__(self, x): 
        for layer in self.layers:
            x = layer(x) 
        return x  
    def parameters(self): 
        return [p in layer for layer in self.layers for p in layer.parameters()]




In [28]:
x = [2.0, 3.0, -1.0]  
n = MLP(3, [4, 4, 1]) 
n(x) 

Value(data = -0.7621436703533037)

In [36]:
xs = [ 
    [2.0, 3.0, -1.0], 
    [3.0,-1.0, 0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets, appears to be binary classification 

ypred = [n(x) for x in xs]

#Calculating the loss function to help tune or measure the neural net to the intended targets 
#We want to minimize the loss 
ypred

[Value(data = -0.7621436703533037),
 Value(data = -0.44923134168981027),
 Value(data = -0.9364600542462126),
 Value(data = -0.7288934810377614)]

In [38]:
loss = ((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss


<generator object <genexpr> at 0x00000169BCCE3B90>

In [ ]:
draw_dot(loss)

AttributeError: 'generator' object has no attribute '_prev'